## Packaging


Once we've made a working program, we'd like to be able to share it with others.

A good cross-platform build tool is the most important thing: you can always
have collaborators build from source.


### Distribution tools

Distribution tools allow one to obtain a working copy of someone else's package.

- Language-specific tools: 
 - python: PyPI,
 - ruby: Ruby Gems, 
 - perl: CPAN,
 - R: CRAN
 
- Platform specific packagers e.g.:
 - `brew` for MacOS, 
 - `apt`/`yum` for Linux or 
 - [`choco`](https://chocolatey.org/) for Windows.

### Laying out a project


When planning to package a project for distribution, defining a suitable
project layout is essential.




In [1]:
%%bash
tree --charset ascii greetings -I "doc|build|Greetings.egg-info|dist|*.pyc"

greetings
|-- greetings
|   |-- command.py
|   |-- greeter.py
|   |-- __init__.py
|   `-- test
|       |-- fixtures
|       |   `-- samples.yaml
|       |-- __init__.py
|       `-- test_greeter.py
|-- CITATION.md
|-- conf.py
|-- index.rst
|-- LICENSE.md
|-- README.md
|-- scripts
`-- setup.py

6 directories, 12 files


We can start by making our directory structure. You can create many nested directories at once using the `-p` switch on `mkdir`.

In [2]:
%%bash
mkdir -p greetings/greetings/test/fixtures
mkdir -p greetings/scripts

### Using setuptools


To make python code into a package, we have to write a `setup.py` file:

```python
from setuptools import setup, find_packages

setup(
    name="Greetings",
    version="0.1.0",
    packages=find_packages(exclude=['*test']),
)
```

We can now install this code with

```
pip install .
```


And the package will be then available to use everywhere on the system.


In [3]:
from greetings.greeter import greet
greet("Terry","Gilliam")

'Hey, Terry Gilliam.'

### Convert the script to a module


Of course, there's more to do when taking code from a quick script and turning it into a proper module:

We need to add docstrings to our functions, so people can know how to use them.

In [4]:
from IPython.display import Code
Code("greetings/greetings/greeter.py")

def greet(personal, family, title="", polite=False):
    """ Generate a greeting string for a person.

    Parameters
    ----------
    personal: str
        A given name, such as Will or Jean-Luc
    family: str
        A family name, such as Riker or Picard
    title: str
        An optional title, such as Captain or Reverend
    polite: bool
        True for a formal greeting, False for informal.

    Returns
    -------
    string
        An appropriate greeting

    Examples
    --------
    >>> from greetings.greeter import greet
    >>> greet("Terry", "Jones")
    'Hey, Terry Jones.
    """

    greeting = "How do you do, " if polite else "Hey, "
    if title:
        greeting += f"{title} "

    greeting += f"{personal} {family}."
    return greeting

In [5]:
import greetings
help(greetings.greeter.greet)

Help on function greet in module greetings.greeter:

greet(personal, family, title='', polite=False)
    Generate a greeting string for a person.
    
    Parameters
    ----------
    personal: str
        A given name, such as Will or Jean-Luc
    family: str
        A family name, such as Riker or Picard
    title: str
        An optional title, such as Captain or Reverend
    polite: bool
        True for a formal greeting, False for informal.
    
    Returns
    -------
    string
        An appropriate greeting
    
    Examples
    --------
    >>> from greetings.greeter import greet
    >>> greet("Terry", "Jones")
    'Hey, Terry Jones.



The documentation string explains how to use the function; don't worry about this for now, we'll consider
this on [the next section](./04documentation.html) ([notebook version](./04documentation.ipynb)).

### Write an executable script

In [6]:
Code("greetings/greetings/command.py")

from argparse import ArgumentParser
from .greeter import greet # Note python 3 relative import

def process():
   parser = ArgumentParser(description="Generate appropriate greetings")

   parser.add_argument('--title', '-t')
   parser.add_argument('--polite', '-p', action="store_true")
   parser.add_argument('personal')
   parser.add_argument('family')

   arguments = parser.parse_args()

   print(greet(arguments.personal, arguments.family, 
               arguments.title, arguments.polite))

if __name__ == "__main__":
    process()

### Specify dependencies

We use the setup.py file to specify the packages we depend on:

```python
setup(
    name="Greetings",
    version="0.1.0",
    packages=find_packages(exclude=['*test']),
    install_requires=['numpy', 'pyyaml'] # NOTE: this is an example to ilustrate how to add dependencies.
)                                        #       Greetings doesn't have any external dependency.
```

### Specify entry point

This allows us to create a command to execute part of our library. In this case when we execute `greet` on the terminal, we will be calling the `process` function under `greetings/command.py`.


In [7]:
Code("greetings/setup.py")

from setuptools import setup, find_packages

setup(
    name="Greetings",
    version="0.1.0",
    packages=find_packages(exclude=['*test']),
    entry_points={
        'console_scripts': [
            'greet = greetings.command:process'
        ]})


And the scripts are now available as command line commands:




In [8]:
%%bash
greet --help

usage: greet [-h] [--title TITLE] [--polite] personal family

Generate appropriate greetings

positional arguments:
  personal
  family

optional arguments:
  -h, --help            show this help message and exit
  --title TITLE, -t TITLE
  --polite, -p


In [9]:
%%bash
greet Terry Gilliam
greet --polite Terry Gilliam
greet Terry Gilliam --title Cartoonist

Hey, Terry Gilliam.
How do you do, Terry Gilliam.
Hey, Cartoonist Terry Gilliam.


### Installing from GitHub


We could now submit "greeter" to PyPI for approval, so everyone could `pip install` it.

However, when using git, we don't even need to do that: we can install directly from any git URL:


```
pip install git+git://github.com/ucl-rits/greeter
```

In [10]:
%%bash
greet Lancelot the-Brave --title Sir

Hey, Sir Lancelot the-Brave.


### Write a readme file

e.g.:

In [11]:
Code("greetings/README.md")

Greetings!
==========

This is a very simple example package used as part of the UCL
[Research Software Engineering with Python](development.rc.ucl.ac.uk/training/engineering) course.

Usage:
    
Invoke the tool with `greet <FirstName> <Secondname>`

### Write a license file

e.g.:

In [12]:
Code("greetings/LICENSE.md")

(C) University College London 2014

This "greetings" example package is granted into the public domain.

### Write a citation file

e.g.:

In [13]:
Code("greetings/CITATION.md")

If you wish to refer to this course, please cite the URL
http://github-pages.ucl.ac.uk/rsd-engineeringcourse/

Portions of the material are taken from [Software Carpentry](http://software-carpentry.org/)

You may well want to formalise this using the [codemeta.json](https://codemeta.github.io/) standard or the [citation file format](http://citation-file-format.github.io/) - these don't have wide adoption yet, but we recommend it.

### Define packages and executables

In [14]:
%%bash
touch greetings/greetings/test/__init__.py
touch greetings/greetings/__init__.py

### Write some unit tests


Separating the script from the logical module made this possible:







In [15]:
Code("greetings/greetings/test/test_greeter.py")

import yaml
import os
from ..greeter import greet

def test_greeter():
    with open(os.path.join(os.path.dirname(__file__),
                           'fixtures',
                           'samples.yaml')) as fixtures_file:
        fixtures = yaml.safe_load(fixtures_file)
        for fixture in fixtures:
            answer = fixture.pop('answer')
            assert greet(**fixture) == answer




Add a fixtures file:







In [16]:
Code("greetings/greetings/test/fixtures/samples.yaml")

- personal: Eric
  family: Idle
  answer: "Hey, Eric Idle."
- personal: Graham
  family: Chapman
  polite: True
  answer: "How do you do, Graahm Chapman."
- personal: Michael
  family: Palin
  title: CBE
  answer: "Hey, CBE Mike Palin."

In [17]:
%%bash --no-raise-error
pytest

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.1.1, py-1.8.0, pluggy-0.12.0
rootdir: /home/dvd/Documents/Work/RSDG/Projects/Teaching/rsd-engineeringcourse/ch04packaging
collected 1 item

greetings/greetings/test/test_greeter.py F                               [100%]

=================================== FAILURES ===================================
_________________________________ test_greeter _________________________________

    def test_greeter():
        with open(os.path.join(os.path.dirname(__file__),
                               'fixtures',
                               'samples.yaml')) as fixtures_file:
            fixtures = yaml.safe_load(fixtures_file)
            for fixture in fixtures:
                answer = fixture.pop('answer')
>               assert greet(**fixture) == answer
E               AssertionError: assert 'How do you d...aham Chapman.' == 'How do you d...aahm Chapman.'
E           

However, this hasn't told us that also the third test is wrong! A better aproach is to parametrize the test as follows:

In [18]:
%%writefile greetings/greetings/test/test_greeter.py
import yaml
import os
import pytest
from ..greeter import greet

def read_fixture():
    with open(os.path.join(os.path.dirname(__file__),
                           'fixtures',
                           'samples.yaml')) as fixtures_file:
        fixtures = yaml.safe_load(fixtures_file)
    return fixtures

@pytest.mark.parametrize("fixture", read_fixture())
def test_greeter(fixture):
    answer = fixture.pop('answer')
    assert greet(**fixture) == answer


Overwriting greetings/greetings/test/test_greeter.py


Now when we run `pytest`, we get a failure per element in our fixture and we know all that fails.

In [19]:
%%bash --no-raise-error
pytest

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.1.1, py-1.8.0, pluggy-0.12.0
rootdir: /home/dvd/Documents/Work/RSDG/Projects/Teaching/rsd-engineeringcourse/ch04packaging
collected 3 items

greetings/greetings/test/test_greeter.py .FF                             [100%]

=================================== FAILURES ===================================
____________________________ test_greeter[fixture1] ____________________________

fixture = {'family': 'Chapman', 'personal': 'Graham', 'polite': True}

    @pytest.mark.parametrize("fixture", read_fixture())
    def test_greeter(fixture):
        answer = fixture.pop('answer')
>       assert greet(**fixture) == answer
E       AssertionError: assert 'How do you d...aham Chapman.' == 'How do you d...aahm Chapman.'
E         - How do you do, Graham Chapman.
E         ?                    -
E         + How do you do, Graahm Chapman.
E         ?                   +

greeti

We can also make pytest to check whether the docstrings are correct by adding the `--doctest-modules` flag:

In [20]:
%%bash --no-raise-error
pytest --doctest-modules

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.1.1, py-1.8.0, pluggy-0.12.0
rootdir: /home/dvd/Documents/Work/RSDG/Projects/Teaching/rsd-engineeringcourse/ch04packaging
plugins: cov-2.8.1
collected 4 items

greetings/greetings/greeter.py F                                         [ 25%]
greetings/greetings/test/test_greeter.py .FF                             [100%]

=================================== FAILURES ===================================
______________________ [doctest] greetings.greeter.greet _______________________
014 
015     Returns
016     -------
017     string
018         An appropriate greeting
019 
020     Examples
021     --------
022     >>> from greetings.greeter import greet
023     >>> greet("Terry", "Jones")
Expected:
    'Hey, Terry Jones.
Got:
    'Hey, Terry Jones.'

/home/dvd/Documents/Work/RSDG/Projects/Teaching/rsd-engineeringcourse/ch04packaging/greetings/greetings/greeter.py:23: Do

### Developer Install


If you modify your source files, you would now find it appeared as if the program doesn't change.

That's because pip install **copies** the files.

If you want to install a package, but keep working on it, you can do:

```
pip install --editable .
```

### Distributing compiled code


If you're working in C++ or Fortran, there is no language specific repository.
You'll need to write platform installers for as many platforms as you want to
support.

Typically:

* `dpkg` for `apt-get` on Ubuntu and Debian
* `rpm` for `yum`/`dnf` on Redhat and Fedora
* `homebrew` on OSX (Possibly `macports` as well)
* An executable `msi` installer for Windows.


#### Homebrew


Homebrew: A ruby DSL, you host off your own webpage

See an [installer for the cppcourse example](http://github.com/jamespjh/homebrew-reactor)

If you're on OSX, do:


```
brew tap jamespjh/homebrew-reactor
brew install reactor
```